In [1]:
import pandas as pd

from fa_tools.PriceIndependent import FundamentalPreProc
from fa_tools.PriceIndependent import FundamentalProc
from fa_tools.PriceIndependent import FundamentalController

In [2]:
raw_fundamental = pd.read_csv("./data/cfs_fundamental.csv", index_col=0)
raw_fundamental.head()

,reprt_date,reprt_code,reprt_year,stock_code,fs_div,account_nm,thstrm_amount,frmtrm_amount,bfefrmtrm_amount
0,2023-09-19,11011,2023,21820,CFS,유동자산,3.712729e+11,3.009129e+11,2.634466e+11
1,2023-09-19,11011,2023,21820,CFS,비유동자산,3.028090e+11,3.279190e+11,3.587054e+11
2,2023-09-19,11011,2023,21820,CFS,자산총계,6.740819e+11,6.288318e+11,6.221520e+11
3,2023-09-19,11011,2023,21820,CFS,유동부채,5.954552e+10,5.008799e+10,4.370744e+10
4,2023-09-19,11011,2023,21820,CFS,비유동부채,3.332828e+10,2.755942e+10,2.706877e+10


In [3]:
fundamental_preproc = FundamentalPreProc(raw_fundamental)
fundamental = fundamental_preproc()
fundamental

,reprt_date,reprt_code,reprt_year,stock_code,factor,first,second,third
0,2023-09-19,11011,2023,021820,current_assets,3.712729e+11,3.009129e+11,2.634466e+11
1,2023-09-19,11011,2023,021820,fixed_assets,3.028090e+11,3.279190e+11,3.587054e+11
2,2023-09-19,11011,2023,021820,total_assets,6.740819e+11,6.288318e+11,6.221520e+11
3,2023-09-19,11011,2023,021820,current_liabilities,5.954552e+10,5.008799e+10,4.370744e+10
4,2023-09-19,11011,2023,021820,fixed_liabilities,3.332828e+10,2.755942e+10,2.706877e+10
...,...,...,...,...,...,...,...,...
1037,2023-03-23,11011,2022,353810,fixed_liabilities,1.993199e+10,8.670197e+09,1.988501e+10
1038,2023-03-23,11011,2022,353810,total_liabilities,1.049993e+11,7.227937e+10,5.578196e+10
1042,2023-03-23,11011,2022,353810,raw_profit,1.555571e+11,1.294578e+11,8.812327e+10
1043,2023-03-23,11011,2022,353810,operation_profit,1.615308e+10,1.677947e+10,1.519806e+10


In [4]:
factor_df = fundamental.loc[:, ["stock_code", "factor", "first", "second", "third"]].melt(
    ["stock_code", "factor"]
)
factor_df

,stock_code,factor,variable,value
0,021820,current_assets,first,3.712729e+11
1,021820,fixed_assets,first,3.028090e+11
2,021820,total_assets,first,6.740819e+11
3,021820,current_liabilities,first,5.954552e+10
4,021820,fixed_liabilities,first,3.332828e+10
...,...,...,...,...
47812,353810,fixed_liabilities,third,1.988501e+10
47813,353810,total_liabilities,third,5.578196e+10
47814,353810,raw_profit,third,8.812327e+10
47815,353810,operation_profit,third,1.519806e+10


In [5]:
stock_codes = set(factor_df["stock_code"])

factor_analysis_list = list()
for stock_code in stock_codes:
    _factor_analysis_df = FundamentalProc(factor_df, stock_code)()
    _factor_analysis_df["stock_code"] = stock_code
    factor_analysis_list.append(_factor_analysis_df)

factor_analysis_df = pd.concat(factor_analysis_list, axis=0)
factor_analysis_df = factor_analysis_df.reset_index(names="factor")
factor_analysis_df.head()

,factor,first,second,third,stock_code
0,current_assets,1.272615e+11,9.353205e+10,7.889550e+10,091580
1,fixed_assets,1.172292e+11,1.071609e+11,8.247182e+10,091580
2,total_assets,2.444906e+11,2.006930e+11,1.613673e+11,091580
3,current_liabilities,1.081889e+11,8.552152e+10,7.887131e+10,091580
4,fixed_liabilities,1.386528e+10,1.372185e+10,1.008584e+10,091580


In [6]:
factor_analysis_df.pivot(
    index=["stock_code"], columns=["factor"], values=["first", "second", "third"]
)

first                                                    \
factor           ALR       CDR       NPR       OER       OPR       TDR   
stock_code                                                               
000020      0.493406  0.332102  0.063424  0.912125  0.087875  0.332102   
000040      0.407153  1.540329 -0.111066  1.053259 -0.053259  1.540329   
000050      0.120067  1.331294  0.011344  0.918055  0.081945  1.331294   
000070      0.384341  0.431934  0.031913  0.960105  0.039895  0.431934   
000080      0.316473  1.458339  0.034761  0.923697  0.076303  1.458339   
...              ...       ...       ...       ...       ...       ...   
950130      0.808313  0.274266  0.338290  0.546167  0.453833  0.274266   
950140      0.701078  0.370160  0.047954  0.934779  0.065221  0.370160   
950170      0.321773  0.555377 -0.399914  1.418730 -0.418730  0.555377   
950190      0.737304  0.089067  0.253486  0.700953  0.299047  0.089067   
950220      0.843460  0.138994  0.000000  0.000000  0.000000  0.138994   

                                                                       ...  \
factor           TER current_assets current_liabilities  fixed_assets  ...   
stock_code                                                             ...   
000020      0.936576   2.275275e+11        7.556234e+10  2.336090e+11  ...   
000040      1.111066   6.074121e+10        9.356145e+10  8.844403e+10  ...   
000050      0.988656   1.498824e+11        1.995375e+11  1.098436e+12  ...   
000070      0.968087   1.747899e+12        7.549770e+11  2.799882e+12  ...   
000080      0.965239   1.054944e+12        1.538467e+12  2.278494e+12  ...   
...              ...            ...                 ...           ...  ...   
950130      0.661710   5.647790e+08        1.548996e+08  1.339346e+08  ...   
950140      0.952046   7.489087e+07        2.772160e+07  3.193160e+07  ...   
950170      1.399914   8.827784e+09        4.902746e+09  1.860702e+10  ...   
950190      0.746514   7.182946e+07        6.397657e+06  2.559225e+07  ...   
950220      0.000000   9.308837e+07        1.293871e+07  1.727650e+07  ...   

               third                                                   \
factor           TER current_assets current_liabilities  fixed_assets   
stock_code                                                              
000020      0.894459   2.227435e+11        6.749400e+10  2.110643e+11   
000040      1.097476   6.097059e+10        8.412293e+10  9.490287e+10   
000050      0.960435   1.173308e+11        2.856408e+11  1.147571e+12   
000070      0.942027   1.475919e+12        6.820490e+11  2.417646e+12   
000080      0.961606   8.993796e+11        1.435593e+12  2.398404e+12   
...              ...            ...                 ...           ...   
950130      0.607445   1.028910e+08        6.400508e+07  5.352832e+07   
950140      0.977532   6.220037e+07        2.847267e+07  3.772346e+07   
950170      7.114446   1.256292e+10        4.994058e+09  2.351996e+10   
950190      0.657092   1.091909e+08        1.183129e+07  3.829877e+06   
950220      0.000000   7.411022e+07        8.270651e+06  1.379872e+07   

                                                                           \
factor     fixed_liabilities    net_profit operation_profit    raw_profit   
stock_code                                                                  
000020          2.363156e+10  2.871510e+10     2.315337e+10  2.720754e+11   
000040          2.190548e+10 -1.148601e+10     1.219024e+09  1.178344e+11   
000050          2.373315e+11  1.302570e+10     2.185400e+10  3.292247e+11   
000070          1.055729e+12  1.432650e+11     1.703360e+11  2.471226e+12   
000080          7.879126e+11  8.662849e+10     1.984786e+11  2.256323e+12   
...                      ...           ...              ...           ...   
950130          1.274441e+07  4.051709e+07     5.822252e+07  1.032137e+08   
950140          1.569162e+07  2.611636e+06     6.668635e+06  1.162392e+08   
950170         

In [7]:
fa_controller = FundamentalController(factor_analysis_df)

factor_conditions = [
    (
        "current_assets",
        lambda x: (x["third"] < x["second"]) & (x["second"] < x["first"]),
    ),
    (
        "raw_profit",
        lambda x: (x["third"] < x["second"]) & (x["second"] < x["first"]),
    ),
    (
        "net_profit",
        lambda x: (0 < x["first"]) & (0 < x["second"]) & (0 < x["third"]),
    ),
    (
        "TDR",  # Total Debt Ratio
        lambda x: (0.2 < x["first"]) & (x["first"] < 0.7),
    ),
]

filtered_stocks = fa_controller.get_filtered_stocks(factor_conditions)
filtered_stocks

{'000070',
 '000320',
 '000500',
 '000660',
 '000720',
 '000850',
 '000860',
 '000990',
 '001120',
 '001440',
 '001680',
 '001790',
 '001800',
 '001820',
 '002030',
 '002200',
 '002240',
 '002310',
 '002810',
 '002840',
 '003030',
 '003160',
 '003220',
 '003570',
 '004000',
 '004370',
 '004430',
 '004590',
 '005010',
 '005290',
 '005850',
 '005930',
 '006120',
 '007590',
 '008060',
 '009160',
 '009290',
 '009780',
 '009970',
 '010040',
 '010120',
 '010130',
 '010470',
 '011170',
 '011210',
 '011370',
 '012330',
 '012630',
 '012750',
 '013890',
 '014580',
 '014830',
 '015230',
 '015890',
 '017650',
 '018260',
 '018670',
 '019770',
 '020000',
 '025530',
 '025540',
 '031330',
 '031430',
 '035510',
 '035890',
 '035900',
 '036540',
 '036890',
 '037350',
 '037460',
 '038540',
 '039420',
 '039610',
 '042370',
 '045060',
 '045100',
 '046140',
 '047040',
 '047770',
 '048430',
 '048910',
 '049070',
 '051600',
 '051910',
 '052460',
 '052690',
 '052790',
 '053260',
 '053620',
 '053690',
 '053800',